In [1]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install selenium


   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   --------------- ------------------------ 3.7/9.5 MB 21.8 MB/s eta 0:00:01
   ---------------------------- ----------- 6.8/9.5 MB 17.5 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 0.58.0
    Uninstalling websocket-client-0.58.0:
      Successfully uninstalled websocket-client-0.58.0


In [17]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time

def get_rendered_html_for_year(url, year):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    
    # Try to find and click the "I Accept" button if the cookie banner appears
    try:
        # Wait for the cookie accept button to appear and click it
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".accept-btn"))
        )
        # Find the "I Accept" button and click it
        accept_button = driver.find_element(By.CSS_SELECTOR, ".accept-btn")
        accept_button.click()
    except TimeoutException:
        # If no cookie banner is found, continue without clicking
        print("No cookie banner found or timeout expired.")
    
    # Wait until the year dropdown is visible
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "cfl-year-button"))
    )
    
    # Click the year dropdown to show the list
    year_button = driver.find_element(By.ID, "cfl-year-button")
    year_button.click()
    
    # Select the year from the dropdown (find the correct <a> element for the year)
    year_option = driver.find_element(By.XPATH, f"//a[text()='{year}']")
    year_option.click()
    
    # Wait for the page to update with press releases for the selected year
    time.sleep(5)  # Adjust based on load time
    
    # After year is selected, scrape the page's content
    rendered_html = driver.page_source
    driver.quit()
    return rendered_html

def scrape_press_releases_by_year(url, year):
    rendered_html = get_rendered_html_for_year(url, year)
    soup = BeautifulSoup(rendered_html, 'html.parser')

    subpage_links = []
    for link in soup.find_all('a', href=True):
        subpage_url = link['href']
        if "/press-releases/" in subpage_url:
            if not subpage_url.startswith('http'):
                subpage_url = requests.compat.urljoin(url, subpage_url)
            subpage_links.append(subpage_url)
    
    return subpage_links

# Scrape all years (use the correct base URL)
url = "https://www.scania.com/se/sv/home/newsroom/press-releases.html"
all_years = [2022, 2023, 2024, 2025]
all_links = []

for year in all_years:
    print(f"Scraping year {year}...")
    links = scrape_press_releases_by_year(url, year)
    all_links.extend(links)

print("All press release links:")
print(all_links)


Scraping year 2022...
Scraping year 2023...
Scraping year 2024...
Scraping year 2025...
All press release links:
['https://www.scania.com/se/sv/home/newsroom/press-releases/press-release-detail-page.html/4414910-scania-sverige-delar-in-aterforsaljarnatverket-i-malardalen-i-tva-regioner', 'https://www.scania.com/se/sv/home/newsroom/press-releases/press-release-detail-page.html/4412872-saljstart-for-nya-avtalsportfoljen-service-360', 'https://www.scania.com/se/sv/home/newsroom/press-releases/press-release-detail-page.html/4408313-forsta-publika-snabbladdarna-for-tunga-transporter-i-norra-sverige', 'https://www.scania.com/se/sv/home/newsroom/press-releases/press-release-detail-page.html/4404503-', 'https://www.scania.com/se/sv/home/newsroom/press-releases/press-release-detail-page.html/4706473-scania-prisas-for-hallbara-ellastbilar', 'https://www.scania.com/se/sv/home/newsroom/press-releases/press-release-detail-page.html/4699437-scanias-nya-biogasmotorer-ger-fem-procents-branslebesparing

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import requests

def get_page_text(url):
    # Open the page using Selenium
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)

    try:
        # Wait until the page is fully loaded (adjust the selector if necessary)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        # Get the page's HTML content
        rendered_html = driver.page_source

        # Parse the page using BeautifulSoup
        soup = BeautifulSoup(rendered_html, 'html.parser')

        # Extract the main content (you can adjust this depending on the page structure)
        content = soup.get_text(separator='\n', strip=True)

        driver.quit()

        return content

    except TimeoutException:
        print(f"Timeout while loading page: {url}")
        driver.quit()
        return None

def scrape_press_release_texts(links):
    press_releases = []

    for link in links:
        print(f"Scraping content from: {link}")
        content = get_page_text(link)

        if content:
            press_releases.append({
                "url": link,
                "content": content
            })

    return press_releases

# Assuming `all_links` contains the list of press release links
# For example:
# all_links = ["https://www.example.com/press-release-1", "https://www.example.com/press-release-2", ...]

press_releases = scrape_press_release_texts(all_links)

def save_to_file(press_releases, filename="scania_press_releases.txt"):
    with open(filename, 'w', encoding='utf-8') as f:
        for release in press_releases:
            f.write(f"URL: {release['url']}\n")
            f.write(f"Content:\n{release['content']}\n\n")
            f.write("-" * 80 + "\n")  # Separator for readability

# Save the press releases to a file
save_to_file(press_releases)

# Print the first press release content (you can modify this to save or process further)
if press_releases:
    print("\nSample Press Release Content:")
    print(press_releases[0]['content'])
    print(len(press_releases)



Scraping content from: https://www.scania.com/se/sv/home/newsroom/press-releases/press-release-detail-page.html/4414910-scania-sverige-delar-in-aterforsaljarnatverket-i-malardalen-i-tva-regioner
Scraping content from: https://www.scania.com/se/sv/home/newsroom/press-releases/press-release-detail-page.html/4412872-saljstart-for-nya-avtalsportfoljen-service-360
Scraping content from: https://www.scania.com/se/sv/home/newsroom/press-releases/press-release-detail-page.html/4408313-forsta-publika-snabbladdarna-for-tunga-transporter-i-norra-sverige
Scraping content from: https://www.scania.com/se/sv/home/newsroom/press-releases/press-release-detail-page.html/4404503-
Scraping content from: https://www.scania.com/se/sv/home/newsroom/press-releases/press-release-detail-page.html/4706473-scania-prisas-for-hallbara-ellastbilar
Scraping content from: https://www.scania.com/se/sv/home/newsroom/press-releases/press-release-detail-page.html/4699437-scanias-nya-biogasmotorer-ger-fem-procents-bransleb

In [28]:
print(len(press_releases))

49
